In [0]:
from keras.datasets import cifar10

import numpy as np
import keras
import tensorflow as tf

from sklearn.model_selection import train_test_split
from keras.layers.normalization import BatchNormalization

random_state = 42

Using TensorFlow backend.


In [0]:
(cifar_X_1, cifar_y_1), (cifar_X_2, cifar_y_2) = cifar10.load_data()

cifar_X = np.r_[cifar_X_1, cifar_X_2]
cifar_y = np.r_[cifar_y_1, cifar_y_2]

cifar_X = cifar_X.astype('float32') / 255
cifar_y = np.eye(10)[cifar_y.astype('int32').flatten()]

train_X, test_X, train_y, test_y = train_test_split(
    cifar_X, cifar_y, test_size=10000, random_state=random_state)
train_X, valid_X, train_y, valid_y = train_test_split(
    train_X, train_y, test_size=10000, random_state=random_state)

170500096/170498071 [==============================] - 6s 0us/step


In [0]:
cifar_X_1.shape

(50000, 32, 32, 3)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, Input, Lambda
from keras.callbacks import EarlyStopping

In [0]:
model = Sequential()

In [0]:
model = Sequential()

# 32x32x3 -> 30x30x64
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu',
                 kernel_initializer='he_normal', input_shape=(32, 32, 3)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu',
                 kernel_initializer='he_normal'))
model.add(MaxPooling2D(pool_size=(2, 2)))  # 28x28x64 -> 14x14x64
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu',
                 kernel_initializer='he_normal'))  # 14x14x64 -> 12x12x128
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu',
                 kernel_initializer='he_normal'))
model.add(MaxPooling2D(pool_size=(2, 2)))  # 12x12x128 -> 5x5x128
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())  # 5x5x16 -> 400
model.add(Dense(120, activation='relu',
                kernel_initializer='he_normal'))  # 400 ->120
# 120 ->84
model.add(Dense(84, activation='relu', kernel_initializer='he_normal'))  
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))  # 84 ->10

In [0]:
model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)

In [0]:
early_stopping = EarlyStopping(patience=1, verbose=1)
model.fit(x=train_X, y=train_y, batch_size=128, epochs=10, verbose=1,
          validation_data=(valid_X, valid_y), callbacks=[early_stopping])

# CIFAR-10 Data Augmentation

In [0]:
from keras.datasets import cifar10

import numpy as np
import keras

from sklearn.model_selection import train_test_split
from keras.layers.normalization import BatchNormalization

random_state = 42

In [0]:
(cifar_X_1, cifar_y_1), (cifar_X_2, cifar_y_2) = cifar10.load_data()

cifar_X = np.r_[cifar_X_1, cifar_X_2]
cifar_y = np.r_[cifar_y_1, cifar_y_2]

cifar_X = cifar_X.astype('float32') / 255
cifar_y = np.eye(10)[cifar_y.astype('int32').flatten()]

train_X, test_X, train_y, test_y = train_test_split(
    cifar_X, cifar_y, test_size=10000, random_state=random_state)
train_X, valid_X, train_y, valid_y = train_test_split(
    train_X, train_y, test_size=10000, random_state=random_state)

In [0]:
cifar_X_1.shape

(50000, 32, 32, 3)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.callbacks import EarlyStopping

In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
datagen = ImageDataGenerator(
    width_shift_range=0.2,   # 左右にずらす
    height_shift_range=0.2,  # 上下にずらす
    horizontal_flip=True,    # 左右反転
    samplewise_center=False,
    samplewise_std_normalization=False,
    zca_whitening=False)

In [0]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3))) # 32x32x3 -> 30x30x64
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu')) # 30x30x64 -> 28x28x64
model.add(MaxPooling2D(pool_size=(2, 2))) # 28x28x64 -> 14x14x64

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu')) # 14x14x64 -> 12x12x128
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu')) # 12x12x128 -> 10x10x128
model.add(MaxPooling2D(pool_size=(2, 2))) # 10x10x128 -> 5x5x128

model.add(Flatten())
model.add(Dense(10, activation='softmax'))

In [0]:
model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)

In [0]:
early_stopping = EarlyStopping(patience=1, verbose=1)
model.fit_generator(datagen.flow(train_X, train_y, batch_size=100),
                    steps_per_epoch=train_X.shape[0] // 100, 
                    epochs=10, validation_data=(valid_X, valid_y))

Epoch 1/10
400/400 [==============================] - 17s 43ms/step - loss: 1.7542 - acc: 0.3508 - val_loss: 1.5001 - val_acc: 0.4679
Epoch 2/10
400/400 [==============================] - 17s 41ms/step - loss: 1.4272 - acc: 0.4811 - val_loss: 1.1155 - val_acc: 0.6004
Epoch 3/10
400/400 [==============================] - 16s 41ms/step - loss: 1.2716 - acc: 0.5460 - val_loss: 1.0241 - val_acc: 0.6426
Epoch 4/10
400/400 [==============================] - 16s 40ms/step - loss: 1.1635 - acc: 0.5869 - val_loss: 0.9918 - val_acc: 0.6559
Epoch 5/10
400/400 [==============================] - 17s 42ms/step - loss: 1.0793 - acc: 0.6184 - val_loss: 0.9189 - val_acc: 0.6821
Epoch 6/10
400/400 [==============================] - 16s 41ms/step - loss: 1.0125 - acc: 0.6435 - val_loss: 0.8874 - val_acc: 0.6947
Epoch 7/10
400/400 [==============================] - 17s 42ms/step - loss: 0.9638 - acc: 0.6612 - val_loss: 0.8236 - val_acc: 0.7100
Epoch 8/10
400/400 [==============================] - 17s 42ms

In [0]:
score = model.evaluate(test_X, test_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.76406021900177
Test accuracy: 0.7452


# Skip connection

In [0]:
from keras.datasets import cifar10

import numpy as np
import keras

from sklearn.model_selection import train_test_split
from keras.layers.normalization import BatchNormalization

random_state = 42

In [0]:
(cifar_X_1, cifar_y_1), (cifar_X_2, cifar_y_2) = cifar10.load_data()

cifar_X = np.r_[cifar_X_1, cifar_X_2]
cifar_y = np.r_[cifar_y_1, cifar_y_2]

cifar_X = cifar_X.astype('float32') / 255
cifar_y = np.eye(10)[cifar_y.astype('int32').flatten()]

train_X, test_X, train_y, test_y = train_test_split(
    cifar_X, cifar_y, test_size=10000, random_state=random_state)
train_X, valid_X, train_y, valid_y = train_test_split(
    train_X, train_y, test_size=10000, random_state=random_state)

In [0]:
cifar_X_1.shape

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.callbacks import EarlyStopping

In [0]:
def resblock(x, filters=64, kernel_size=(3, 3)):
    x_ = Conv2D(filters, kernel_size, padding='same')(x)
    x_ = BatchNormalization()(x_)
    x_ = Conv2D(filters, kernel_size, padding='same')(x_)
    x = Add()([x_, x])
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

from keras import backend as K
from keras.layers import Add

In [0]:
inputs = Input(shape=(32, 32, 3))

x = Conv2D(64, kernel_size=(5, 5), padding='same', activation='relu')(inputs)
x = resblock(x)
x = resblock(x)
x = Flatten()(x)
y = Dense(10, activation='softmax')(x)

In [0]:
model = Model(inputs=inputs, outputs=y)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
early_stopping = EarlyStopping(patience=1, verbose=1)
model.fit(x=train_X, y=train_y, batch_size=128, epochs=10, verbose=1,
          validation_data=(valid_X, valid_y), callbacks=[early_stopping])

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 16s 410us/step - loss: 1.2621 - acc: 0.5545 - val_loss: 1.4116 - val_acc: 0.5110
Epoch 2/10
40000/40000 [==============================] - 16s 411us/step - loss: 1.1010 - acc: 0.6137 - val_loss: 1.1659 - val_acc: 0.5908
Epoch 3/10
40000/40000 [==============================] - 16s 411us/step - loss: 0.9606 - acc: 0.6665 - val_loss: 1.1646 - val_acc: 0.5908
Epoch 4/10
40000/40000 [==============================] - 16s 409us/step - loss: 0.8392 - acc: 0.7110 - val_loss: 1.3000 - val_acc: 0.5885
Epoch 00004: early stopping


# Cifer10 転移学習 VGG16

In [0]:
from keras.datasets import cifar10

import numpy as np
import keras

from sklearn.model_selection import train_test_split
from keras.layers.normalization import BatchNormalization

random_state = 42

In [0]:
(cifar_X_1, cifar_y_1), (cifar_X_2, cifar_y_2) = cifar10.load_data()

cifar_X = np.r_[cifar_X_1, cifar_X_2]
cifar_y = np.r_[cifar_y_1, cifar_y_2]

cifar_X = cifar_X.astype('float32') / 255
cifar_y = np.eye(10)[cifar_y.astype('int32').flatten()]

train_X, test_X, train_y, test_y = train_test_split(
    cifar_X, cifar_y, test_size=10000, random_state=random_state)
train_X, valid_X, train_y, valid_y = train_test_split(
    train_X, train_y, test_size=10000, random_state=random_state)

In [74]:
cifar_X_1.shape

(50000, 32, 32, 3)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.callbacks import EarlyStopping

In [0]:
from keras.applications import VGG16,InceptionV3
from keras.layers import UpSampling2D, Input
from keras.models import Model

In [0]:
image_size = 128
num_classes = 10

# モデルの定義
input = Input((32,32,3))
x = UpSampling2D(4)(input)
x
model = InceptionV3(weights='imagenet', include_top=False, input_tensor=x)

top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(num_classes, activation='softmax'))

model = Model(inputs=model.input, outputs=top_model(model.output))

# model.summary()
for layer in model.layers[:15]:
    layer.trainable = False

In [78]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_25 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
up_sampling2d_17 (UpSampling2D) (None, 128, 128, 3)  0           input_25[0][0]                   
__________________________________________________________________________________________________
conv2d_204 (Conv2D)             (None, 63, 63, 32)   864         up_sampling2d_17[0][0]           
__________________________________________________________________________________________________
batch_normalization_195 (BatchN (None, 63, 63, 32)   96          conv2d_204[0][0]                 
__________________________________________________________________________________________________
activation

In [0]:
model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)

In [80]:
early_stopping = EarlyStopping(patience=1, verbose=1)
model.fit(x=train_X, y=train_y, batch_size=128, epochs=10, verbose=1,
          validation_data=(valid_X, valid_y), callbacks=[early_stopping])

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 116s 3ms/step - loss: 1.5609 - acc: 0.4817 - val_loss: 3.0398 - val_acc: 0.3131
Epoch 2/10
40000/40000 [==============================] - 92s 2ms/step - loss: 1.1318 - acc: 0.6553 - val_loss: 2.5872 - val_acc: 0.2896
Epoch 3/10
40000/40000 [==============================] - 92s 2ms/step - loss: 0.9740 - acc: 0.7069 - val_loss: 13.4136 - val_acc: 0.0927
Epoch 00003: early stopping
